## New York City Emissions Intensity Map
##### Notebook by [Mohamad Razkenari](https://www.razkenari.com)

<img src="https://www.urbangreencouncil.org/sites/default/files/landing_page_image.jpg"/>

## Introduction
In the time it took you to read this sentence, terabytes of data have been collectively generated across the world — more data than any of us could ever hope to process, much less make sense of, on the machines we're using to read this notebook.

<img src="https://www.circuitmeter.com/wp-content/uploads/2020/02/image-industryArticles_NYC-LL97.jpg" width=50% />


>**Note:** Please specify.

Here’s what you need to know

* Applies to all buildings in NYC greater than 25000 square feet; applicable for approximately 60% residential and 40% commercial buildings
* Penalties for non-compliance; maximum penalty is calculated based on the difference between a building’s annual emissions limit and its actual emissions multiplied by 268
* A building’s annual emissions limit equals its emissions intensity limit multiplied by its gross floor area
* Many buildings are significantly above emissions limits and will require comprehensive retrofits or potentially face millions of dollars in penalties
* There are two increasingly stringent phases of limits; first phase takes effect in 2024 and the second occurring in 2030
* Emissions intensity limits (metric tons of CO2e per square foot) are set based on 10 building categories defined by Building Code occupancy groups
* Renewable energy credits and/or emissions offsets can be used to comply with the caps

## Required Datasets

**Step 1 - Calculate Emissions Intensity**: total global warming potential per square foot per year, neasured in kg CO2e/m2/a
* Energy Consumption per year
 * Natural Gas
 * Electricity
 * District Steam
* Building Floor area 

**Step 2 - Identify Emissions Target**: Based on building classification, as specified by the Local Law 97
* Building Occupancy Classification
* Global Warming Potential per unit of energy

**Data Sources:**
* **Energy use benchmarking data** (required by LL84) [LINK](https://data.cityofnewyork.us/Environment/Local-Law-84-2021-Monthly-Data-for-Calendar-Year-2/in83-58q5) 
* **Building footprint data** [LINK](https://data.cityofnewyork.us/Housing-Development/Building-Footprints/nqwf-w8eh)
* **Building 3D models** [LINK](https://www1.nyc.gov/site/doitt/initiatives/3d-building.page)
* **Building Use and Characteristics** [LINK](https://www1.nyc.gov/site/planning/data-maps/open-data/bytes-archive.page?sorts[year]=0)

** Featureset Exploration **

* **age**: continuous. 
* **workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
* **education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. 
* **education-num**: continuous. 
* **marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. 
* **occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. 
* **relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. 
* **race**: Black, White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other. 
* **sex**: Female, Male. 
* **capital-gain**: continuous. 
* **capital-loss**: continuous. 
* **hours-per-week**: continuous. 
* **native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

## Required libraries
This notebook uses several Python packages that come standard with the Anaconda Python distribution. The primary libraries that we'll be using are:

* **NumPy**: Provides a fast numerical array structure and helper functions.
* **Pandas**: Provides a DataFrame structure to store data in memory and work with it easily and efficiently.
* **Scikit-learn**: The essential Machine Learning package in Python.
* **Matplotlib**: Basic plotting library in Python; most other Python plotting libraries are built on top of it.
* **Seaborn**: Advanced statistical plotting library.
* **ArcGIS**: Advanced library to convert and manage geographic data, automate spatial workflows, perform advanced spatial analytics.
* **Sodapy**: API for getting NYC data.
* **ArcPy**: ArcPy is a site-package that builds on the successful arcgisscripting module.

In [59]:
# Install and Import packages in the current Jupyter kernel
import sys
import os
# !{sys.executable} -m pip install numpy pandas scikit-learn matplotlib seaborn arcgis sodapy
 
import pandas as pd
import numpy as np
from sodapy import Socrata

from arcgis import GIS
from arcgis.geocoding import geocode
from arcgis.geometry import lengths, areas_and_lengths, project
from arcgis.geometry import Point, Polyline, Polygon, Geometry

In [43]:
# Enter the information from those sections here
socrata_domain = 'data.cityofnewyork.us'

consumption_id = 'in83-58q5' #building consumption
footprint_id = 'qb5r-6dgf' #building foorprint coordinates
points_id = '7w4b-tj9d' #building coordinates

socrata_token = os.environ.get("SODAPY_APPTOKEN")
client = Socrata(socrata_domain, socrata_token)

In [45]:
consumption = client.get(consumption_id)
# consumption = client.get_all(consumption_id)
df_consumption = pd.DataFrame.from_dict(consumption)
df_consumption.head()

,property_id,property_name,parent_property_id,parent_property_name,month,electricity_use_kbtu,natural_gas_use_kbtu
0,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Jan-20,2175731.1,Not Available
1,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Feb-20,1902208.9,Not Available
2,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Mar-20,1847793.7,Not Available
3,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,Apr-20,1613573.0,Not Available
4,7365,1155,Not Applicable: Standalone Property,Not Applicable: Standalone Property,May-20,1747078.0,Not Available


In [46]:
# points_client = Socrata(socrata_domain, socrata_token)
points = client.get(points_id)
# points = client.get_all(points_id)
df_points = pd.DataFrame.from_dict(points)
df_points.head()

,the_geom,bin,cnstrct_yr,lstmoddate,lststatype,doitt_id,heightroof,feat_code,groundelev,base_bbl,mpluto_bbl,geomsource
0,"{'type': 'Point', 'coordinates': [-73.96673233...",3170958,1925,2017-08-22T00:00:00.000,Constructed,96807,29.74985318,2100,40,3065220021,3065220021,Photogramm
1,"{'type': 'Point', 'coordinates': [-74.16795096...",5028452,1965,2017-08-22T00:00:00.000,Constructed,326368,22.63,2100,39,5012640036,5012640036,Photogramm
2,"{'type': 'Point', 'coordinates': [-74.19517887...",5078368,1970,2017-08-22T00:00:00.000,Constructed,746627,35.76,2100,51,5060190091,5060190091,Photogramm
3,"{'type': 'Point', 'coordinates': [-73.96116485...",3245111,1928,2017-08-22T00:00:00.000,Constructed,786626,37.5,2100,6,3086910048,3086910048,Photogramm
4,"{'type': 'Point', 'coordinates': [-73.75428671...",4161096,1950,2017-08-22T00:00:00.000,Constructed,746409,18.01511294,2100,93,4075020005,4075020005,Photogramm


In [47]:
footprint = client.get(footprint_id)
# footprint = client.get_all(footprint_id) # to get all data
df_footprint = pd.DataFrame.from_dict(footprint)
df_footprint.head()

,the_geom,bin,cnstrct_yr,lstmoddate,lststatype,doitt_id,heightroof,feat_code,groundelev,shape_area,shape_len,base_bbl,mpluto_bbl,geomsource
0,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",3170958,1925,2017-08-22T00:00:00.000,Constructed,96807,29.74985318,2100,40,0.0,0.0,3065220021,3065220021,Photogramm
1,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",5028452,1965,2017-08-22T00:00:00.000,Constructed,326368,22.63,2100,39,0.0,0.0,5012640036,5012640036,Photogramm
2,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",5078368,1970,2017-08-22T00:00:00.000,Constructed,746627,35.76,2100,51,0.0,0.0,5060190091,5060190091,Photogramm
3,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",3245111,1928,2017-08-22T00:00:00.000,Constructed,786626,37.5,2100,6,0.0,0.0,3086910048,3086910048,Photogramm
4,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",4161096,1950,2017-08-22T00:00:00.000,Constructed,746409,18.01511294,2100,93,0.0,0.0,4075020005,4075020005,Photogramm


In [100]:
a = df_points['the_geom'][0]['coordinates']
a

[-73.96673233583692, 40.626022404070476]

In [156]:
gis = GIS()
m=gis.map('New York, NY')
m

MapView(layout=Layout(height='400px', width='100%'))

In [157]:
pt = Point({"x" : a[0], "y" : a[1], 
            "spatialReference" : {"wkid" : 4326}})
pt.is_valid()

True

In [158]:
# m.draw(pt, symbol=pt_sym)

In [159]:
l = df_footprint['the_geom'][0]['coordinates'][0]
l

[[[-73.96664570466969, 40.62599676998366],
  [-73.96684846176461, 40.625977490862574],
  [-73.96685938726297, 40.62604419372411],
  [-73.96661621040211, 40.62606731716107],
  [-73.96660638332114, 40.626007324369795],
  [-73.96664680403327, 40.626003480977275],
  [-73.96664570466969, 40.62599676998366]]]

In [160]:
line1 = {
  "paths" : l,
  "spatialReference" : {"wkid" : 4326}
}
polyline1 = Polyline(line1)
print(polyline1.is_valid())

True


In [162]:
sym_poly_aoi = {
  "type": "esriSFS",
  "style": "esriSFSSolid",
  "color": [255,140,0,255],
    "outline": {
     "type": "esriSLS",
     "style": "esriSLSSolid",
     "color": [0,255,0,255],
     "width": 3}
}
m.draw(polyline1, symbol = sym_poly_aoi)

In [145]:
# polygon1 = Polygon({'spatialReference': {'latestWkid': 4326}, 
#                 'rings': l
#                 })

In [148]:
# polygon1

In [149]:
# m.draw(polygon1)

In [98]:
def convert_to_rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)    
    halfmax = (minimum + maximum) / 2
    if minimum <= value <= halfmax:
        r = 0
        g = int( 255./(halfmax - minimum) * (value - minimum))
        b = int( 255. + -255./(halfmax - minimum)  * (value - minimum))
        return (r,g,b)    
    elif halfmax < value <= maximum:
        r = int( 255./(maximum - halfmax) * (value - halfmax))
        g = int( 255. + -255./(maximum - halfmax)  * (value - halfmax))
        b = 0
        return (r,g,b)

In [163]:
# https://geopandas.org/en/stable/gallery/choro_legends.html

# https://geopandas.org/en/stable/getting_started/introduction.html

In [97]:
# import httplib, urllib, base64

# headers = {
#     # Request headers
#     'Ocp-Apim-Subscription-Key': '{subscription key}',
# }

# params = urllib.urlencode({
# })


# import geopandas as gpd
# import matplotlib.pyplot as plt 
# from mpl_toolkits.axes_grid1 import make_axes_locatable 

# # Reading the world shapefile 
# world_data = gpd.read_file(r'D:\GeoDelta\Introduction to Geopandas_Basics\Visualizing Geographical Data\world.shp')
# world_data = world_data[['NAME', 'geometry']]

# # Calculating the area of each country 
# world_data['area'] = world_data.area

# # Removing Antarctica from GeoPandas GeoDataframe
# world_data = world_data[world_data['NAME'] != 'Antarctica']
# world_data.plot()

# # Changing the projection
# current_crs = world_data.crs   
# world_data.to_crs(epsg = 3857, inplace = True)
# world_data.plot(column = 'NAME', cmap = 'hsv')

# # Re-calculate the areas in Sq. Km.
# world_data['area'] = world_data.area/1000000

# # Adding a legend 
# world_data.plot(column = 'area' , cmap = 'hsv' , legend = True, 
#                 legend_kwds = {'label': "Area of the country (Sq. Km.)"}, figsize = (7,7))

# # Resizing the legend 
# fig, ax = plt.subplots(figsize = (10,10))
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size = "7%", pad = 0.1)
# world_data.plot(column = 'NAME' , cmap = 'hsv' , legend = None, 
#                 legend_kwds = {'label': "Area of the country (Sq. Km.)"},
#                 ax = ax, cax = cax)

# https://data.cityofnewyork.us/resource/7w4b-tj9d.json

# https://www1.nyc.gov/site/planning/data-maps/open-data/bytes-archive.page?sorts[year]=0
# https://www1.nyc.gov/site/doitt/initiatives/3d-building.page
# https://www1.nyc.gov/html/gbee/html/plan/ll84_scores.shtml

# https://benchmarking.cityofnewyork.us/#/

# https://uil.carto.com/api/v2/sql?format=csv&filename=filtered_evt&q=SELECT%20bbl,%20bin,%20address,%20zipcode,%20bldgtype,%20ess,%20year,%20gfa,%20numbldgs,%20numfloors,%20eui,%20energy,%20wui,%20ghg,%20primary_type,%20other_types_dict,%20has_datacenter,%20electricity,%20bedrooms,%20natural_gas,%20steam,%20worker_dens,%20res_units,%20diesel,%20fuel_4,%20fuel_2,%20fuel_5_6%20FROM%20table_2017_disclosure_ll84_viz_01_2019%20WHERE%20(year%20%3E=%201900%20AND%20year%20%3C=%202017)%20AND%20(gfa%20%3E=%2010000%20AND%20gfa%20%3C=%20500035)%20%20AND%20(eui%20%3E=%2050%20AND%20eui%20%3C=%20350)